# CHEF ASSISTANT

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

In [3]:
tavily_client = TavilyClient()

### Define tools

In [4]:
@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage
from pprint import pprint

In [6]:
system_prompt= """

You are a Chef assistant who is going to receive some ingredients and search for some recipees I can make with them.

Using the web search tool, search the web for recipes that can be made with the ingredients they have.

Return recipe suggestions and eventually the recipe instructions to the user, if requested.

"""

### Text input

In [7]:
agent = create_agent(
    model='gpt-5-nano',
    tools = [web_search],
    system_prompt = system_prompt,
    checkpointer=InMemorySaver(),
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="I have some leftover chicken and rice. What can I make?")]},
    config
)

In [9]:
print(response['messages'][-1].content)

pprint(response)

Great idea to use leftovers. Here are a few recipe ideas that work well with leftover chicken and rice (with quick notes on flavor and what you’ll usually need). I can pull full instructions for any of these if you’d like.

- Chicken fried rice (one-pan dinner)
  - Why it’s good: Quick, versatile, and uses what you already have. You typically sauté some onions and veggies, toss in the chicken and rice, and finish with soy sauce or tamari.
  - Common extras: eggs, peas, carrots, green onions.

  Source idea: a classic “Chicken Fried Rice” recipe from Quick Weeknight Meals.

- Leftover chicken and egg fried rice
  - Why it’s good: Similar to fried rice but adds eggs for protein and a richer texture. Great for using up leftover roast or baked chicken.
  - Common extras: eggs, soy sauce, peas, scallions.

  Source idea: Leftover Chicken and Egg Fried Rice from Easy Peasy Foodie.

- Easy Chicken and Rice Casserole
  - Why it’s good: Comforting, creamy bake that uses chicken and rice with a 

### Image input

In [8]:
from ipywidgets import FileUpload
from IPython.display import display

In [9]:
uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [10]:
print(uploader.value)

({'name': 'left_overs.png', 'type': 'image/png', 'size': 379961, 'content': <memory at 0x10f9c5a80>, 'last_modified': datetime.datetime(2026, 1, 8, 17, 46, 10, tzinfo=datetime.timezone.utc)},)


In [11]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [13]:
multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "List the ingredients I have"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

response = agent.invoke(
    {"messages": [multimodal_question]},
    config
)

In [14]:
print(response['messages'][-1].content)

Here’s what I can confidently identify from the photo:

- Lemons (two halves visible)
- Fresh greens/lettuce in a bag
- Eggs (carton)
- A bunch of small yogurt cups (several, look like Activia/YoPRO-style yogurts)
- Coca‑Cola can (drink)
- Tortillas or wraps (pack on a shelf)
- Frozen vegetables bag (bottom drawer area)
- Jar of jam/marmalade (mermelada) on the door
- A few bottles/jars on the door shelves (condiments or sauces; not all are clearly identifiable)

Items I can’t be certain about (could be extra pantry/fridge staples tucked in): some sauces/oils, additional dairy, or meat in the lower drawers.

Want me to propose 2–3 recipe ideas using just these identified items? If you have any extra pantry staples (oil, garlic, onions, cheese, etc.), I can tailor more options.
